In [67]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms, datasets
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.backends.cudnn as cudnn

In [83]:
import matplotlib.pyplot as plt
import numpy as np
import copy
import time
import os

In [3]:
import utils
from models import ResNet

In [4]:
# not working

def show_images(image):
    image = image / 2 + 0.5
    npim = image.numpy()
    X = np.transpose(npim, [0, 2, 3, 1])
    plt.imshow(X)

In [5]:
data_loaders, test_loader = utils.get_data_loaders()

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [6]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [68]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.is_available()

if torch.cuda.is_available():
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

In [ ]:
net = ResNet.ResNet50()
net = net.to(device)

print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

In [71]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=32)

Size of CIFAR-10

In [72]:
dataset_size = {'train': 4000,'val': 1000,'test': 1000}

In [101]:
def train_model(model, criterion, optimizer, scheduler, num_epochs):
    
    start = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_accuracy = 0.0
    
    for epoch in range(num_epochs):
        
        print(str(epoch) + "/" + str(num_epochs))
        
        for phase in ['train', 'val']:
            
            print(phase)
            
            if phase == 'train':
                
                scheduler.step()
                model.train()
                
            else:
                model.eval()
            
            running_loss = 0.0
            running_corrects = 0
            total = 0
            
            for index, (inputs, targets) in enumerate(data_loaders[phase]):
                
                inputs = inputs.to(device)
                targets = targets.to(device)

                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = outputs.max(1)
                    loss = criterion(outputs, targets)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                running_loss += loss.item()
                running_corrects += preds.eq(targets).sum().item()
                total += targets.size(0)
                
            epoch_loss = running_loss / total
            epoch_acc = running_corrects / total
                
            print('Loss: ' + str(epoch_loss) + ", Epoch Accuracy: " + str(epoch_acc))
                
            if phase == 'val' and epoch_acc > best_accuracy:
                best_accuracy = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                
                if not os.path.isdir('trained-models'):
                    os.mkdir('trained-models')
                
                state = {
                    
                    'epoch': epoch,
                    'state_dict': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'scheduler': scheduler.state_dict()
                    
                }
                
                torch.save(state, './state/state.t7')
            
    train_time = time.time() - start
    
    print('Best Accuracy: ' + best_accuracy)
    
    model.load_state_dict(best_model_wts)
    return model

In [102]:
epochs = 25

net = train_model(net, criterion, optimizer, scheduler, epochs)

0/25
train
Loss: 0.09365264156125486, Epoch Accuracy: 0.668675
val
Loss: 0.08278618874177336, Epoch Accuracy: 0.7043


FileNotFoundError: [Errno 2] No such file or directory: './state/state.t7'